In [28]:
%load_ext autoreload
%autoreload 2
from solution.model import * 
from solution.romanizer import *
from solution.language import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
pronunciation_tokenizer = CharacterTokenizer.from_pretrained("models/base")
print(pronunciation_tokenizer.get_vocab())

vocab = pronunciation_tokenizer.get_vocab()

phonetics_model = PhoneticTransformer(vocab, pikl="models/base/model_params.pkl")
phonetics_model.load_state_dict(torch.load("models/base/base_v1"))
phonetics_model.to_device("cpu")
phonetics_model.eval()

{'[CLS]': 0, '[SEP]': 1, '[BOS]': 2, '[MASK]': 3, '[PAD]': 4, '[UNK]': 5, '[EOS]': 6, '.': 7, 'ˈ': 8, 't': 9, 'i': 10, 'a': 11, 's': 12, 'n': 13, 'e': 14, 'k': 15, 'o': 16, 'l': 17, 'u': 18, 'ː': 19, 'm': 20, ' ': 21, 'r': 22, 'p': 23, 'ɪ': 24, 'ə': 25, 'ʲ': 26, 'd': 27, '⁵': 28, 'j': 29, 'ɑ': 30, 'ɛ': 31, '͡': 32, '²': 33, '̯': 34, '¹': 35, 'ɔ': 36, 'ɐ': 37, '³': 38, 'ˌ': 39, '̞': 40, 'ʁ': 41, 'b': 42, 'v': 43, '⁴': 44, '̪': 45, 'ɾ': 46, 'f': 47, 'z': 48, '⁻': 49, 'ʊ': 50, 'ɡ': 51, '̃': 52, 'ɨ': 53, 'ɕ': 54, 'ŋ': 55, '̠': 56, 'ʃ': 57, '̝': 58, 'ʰ': 59, ')': 60, '(': 61, 'w': 62, 'h': 63, 'ʂ': 64, '˧': 65, 'y': 66, 'ä': 67, 'æ': 68, 'x': 69, 'ʒ': 70, '̚': 71, 'ɫ': 72, 'ʔ': 73, 'ʋ': 74, '˨': 75, '˩': 76, 'ɲ': 77, '˦': 78, 'ð': 79, 'ʐ': 80, 'ẽ': 81, '̟': 82, 'ɒ': 83, 'ɹ': 84, 'θ': 85, 'ø': 86, 'ĩ': 87, '̩': 88, 'ʈ': 89, '˥': 90, '⁽': 91, '⁾': 92, 'ᵝ': 93, 'β': 94, 'õ': 95, 'ɦ': 96, 'ɯ': 97, 'ɤ': 98, 'ɣ': 99, 'ã': 100, 'ˣ': 101, 'ʝ': 102, 'ʌ': 103, 'ɵ': 104, 'ɻ': 105, 'ʑ': 106, 'ʎ': 107, 

C:\Users\Joaquin\AppData\Local\Temp\ipykernel_20068\3745097377.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  phonetics_model.load_state_dict(torch.load("models/base/ba

PhoneticTransformer(
  (gpts): ModuleList(
    (0-22): 23 x GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(149, 64)
        (wpe): Embedding(29, 64)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-3): 4 x GPT2Block(
            (ln_1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2SdpaAttention(
              (c_attn): Conv1D()
              (c_proj): Conv1D()
              (attn_dropout): Dropout(p=0.1, inplace=False)
              (resid_dropout): Dropout(p=0.1, inplace=False)
            )
            (ln_2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
            (mlp): GPT2MLP(
              (c_fc): Conv1D()
              (c_proj): Conv1D()
              (act): NewGELUActivation()
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
        )
        (ln_f): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
      (lm_head): Linear(in_feature

In [22]:
def generate(model, pad_token_id, top_k = 50, top_p = 0.95, language_temps = None, temperature = 1.0):
    input_ids = pronunciation_tokenizer.encode("").ids[:-1]
    output = model.generate(
        input_ids,
        pad_token_id = pad_token_id,
        no_repeat_ngram_size=0,  # Prevent repetition
        do_sample = True,
        top_k=top_k,  # Limits the sampling pool to top_k tokens
        top_p=top_p,  # Cumulative probability for nucleus sampling
        language_temps = language_temps,
        temperature=temperature,  # Adjust the randomness of predictions,
    )
    output = pronunciation_tokenizer.decode(output[0][1:-1].cpu().numpy())
    return output

In [23]:
GENERATED_LANGUAGE_SIZE = 1000

In [24]:
languages = {
    "English": 1,
}
temps = {
    "German": 0.2
}

phonetics_model.set_language_weights(languages)
language_sounds = []
for _ in range(GENERATED_LANGUAGE_SIZE):
    o = generate(phonetics_model, vocab["[PAD]"], top_k=50, top_p=0.95, language_temps = temps, temperature=1.0)
    language_sounds.append(o)

In [25]:
language_vocabulary = {}
for x in language_sounds:
    y = romanize(x)
    language_vocabulary[x] = y
    print(y, x)

lëz lɜːz
hyumuraz hjʊˈmʊɹəz
sawrhaika ˌsɑː(ɹ) ˈhaɪkə
sleek sliːk
jookl juːkl̩
inmăndafail ɪnˈmɒndəˌfaɪl
bilyurin ˈbɪljəɹɪn
ensжьórsh ɛnˈsʑɔː(ɹ)ʃ
kloond kluːnd
rektrl ˈɹɛk.tɹl̩
gawkkéi ˈɡɑk.keɪ
sawlrănglee ˈsɑl̯ɾɒŋɫi
mikawshn məˈkɑː.ʃn̩
pilee ˈpɪli
kawliz ˈkɑlɪz
rrniuns ˈɹɝnɪəns
æbrelee ˈæb.ɹə.li
dawrkilee ˈdɑː(ɹ)kəli
saimóndzhit saɪˈmɔnd͡ʒət
sooshéi ˈsuːˌʃeɪ
siv səːv
pilidiuii ˌpɪl.ɪ.ˈdəʊ.ɪə
prireenas pɹɪˈɹiːnəs
bindzhil ˈbɪnd͡ʒɪɫ
taineréishtun ˌtaɪ.nə.ˈɹeɪ.ʃtən
vawnsitóth ˈvɑnsɪtɔːθ
dzhin d͡ʒɪn
wéidiz ˈweɪdɪz
deejakefshin ˈdiː jəˈkɛfʃən
seeree ˈsiː.ɹi
dnoosausól ˌdnus.ə(ʊ) sɔl
semeea ˈsɛm.i.ə
mænyoothintarr ˈmænjuθ ˈɪnt(ə)r
koonrizauléishon kuˌnɹɪzəʊˈleɪʃən
di dɪ-
krænlivitawkshon ˌkɹæn.lɪv.əˈtɑk.ʃ.ən
fendalee ˈfɛndəli
benting ˈbɛntɪŋ
prklit ˈpɚklɪ̯t
kwem kwɛm
bouking ˈbəʊkɪŋ
slyootădid ˈsljuˈtɒdɪd
péistl ˈpeɪstl̩
igskraitoulm ˌɪɡskɹaɪˈtoʊlm̩
noontzhawr ˈnunt.ˈʒɑɹ
skr ˈskɝ-
sænstréidiv sænˈstɹeɪdɪv
tesilon ˈtɛsɪlən
mërkeniun ˈmɜː(ɹ)kənɪən
insverael ɪnˈsvɛɹə(ɛl)
pëk pɜːk
rrimfon ˈrɪmfə

In [30]:
language = Language(language_vocabulary= language_vocabulary)
save_language(language, "language")

# Semantics Module

In [31]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

## Train a custom tokenizer

In [46]:
from solution.language import train_custom_tokenizer


custom_tokenizer_path = "custom_tokenizer"
custom_tokenizer = train_custom_tokenizer(language, 50000, custom_tokenizer_path)
print(custom_tokenizer.get_vocab_size())


3166


## Generate new Data

In [32]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-xl")
model = GPT2LMHeadModel.from_pretrained("gpt2-xl")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

c:\Users\Joaquin\miniconda3\envs\agi\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Joaquin\.cache\huggingface\hub\models--gpt2-xl. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

c:\Users\Joaquin\miniconda3\envs\agi\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [35]:
prompt = "Hello World"

In [36]:
input_ids = tokenizer.encode(prompt, return_tensors = "pt")
model.to("cuda")
output = model.generate(
    input_ids, 
    max_length=200,
    num_return_sequences=1,
    temperature = 0.7,
    top_k = 50, 
    top_p = 0.95, 
    do_sample = True 
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [38]:
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

Hello World" to the command line with no parameters, and then you'll be able to run the code, which will print out the output.

I hope you enjoyed this tutorial, and I'll be sure to write more about it in the future. In the meantime, if you have any questions, please leave a comment below.


In [44]:
language_generated_text = custom_tokenizer.decode(output[0].tolist(), skip_special_tokens=True)
print(language_generated_text)

mikàil man dn taizherb krut bawk diz f ov hæps bu yór yu tran man sek dn bsuunvee f krit ætr shur dn h sl sl ġ æmilipieenee bu nas f ov ëm yór yu aridhil man ænd arr ast nór dn glisee h peee dn f ber bu nou bin séino f rileemiezeu alee æmoroumin oves h
